In [63]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [159]:
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\VARTUL\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\VARTUL\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [65]:
ls = list(brown.tagged_sents(tagset = 'universal'))


In [66]:
words = list(brown.words())

In [67]:
len(words)

1161192

In [160]:
#add start and end tags:
def add_start_end_tag(sentences_original):
    sentences=[""]*len(sentences_original)
    for i in range(len(sentences_original)):
        sentences[i]=[("<start>","<start>")]+sentences_original[i]+[("<end>","<end>")]
    return sentences

In [161]:
#given a tag, what is its index : tags dict {"tag" : "tag_index"}
#given an index what is the tag : tags_reverse list of string
#count of each tag: tag_count
def creat_tags_meta(sentences):
    tags_dict={}
    tags_reverse=[""]*100000
    cnt = 0
    tag_count=[]
    for i in range(len(sentences)):
        for j in sentences[i]:
            if j[1] not in tags_dict:
                tags_dict[j[1]] = cnt
                tags_reverse[cnt]=j[1]
                tag_count.append(0)
                cnt += 1
            tag_count[tags_dict[j[1]]]+=1

    return (tags_dict, tags_reverse, tag_count)


In [162]:
#creating transition matrix
def create_transition_matrix(sentences, tags_dict, tag_count):
    bigram_matrix = []
    transition_matrix = []
    total_tags = len(tag_count)
    for i in range(total_tags): 
        bigram_matrix.append([0]*total_tags)
        transition_matrix.append([0]*total_tags)

    #creating a bigram matrix
    for sentence in sentences:
        for j in range(len(sentence)-1):
            bigram_matrix[tags_dict[sentence[j][1]]][tags_dict[sentence[j+1][1]]]+=1
    # print(bigram_matrix)
    
    #create transition matrix A
    for i in range(total_tags):
        for j in range(total_tags):
            transition_matrix[i][j]=bigram_matrix[i][j]/tag_count[i]
    # print(transition_matrix)
    
    return (transition_matrix, bigram_matrix)


In [163]:
#word to index number
def create_word_dict(sentences):

    idx = 0
    word_dict={}
    # word_dict["<start>"]=idx
    for sentence in sentences:
        for tup in sentence:
            if tup[0] not in word_dict:
                
                word_dict[tup[0]]=idx
                idx += 1
    word_dict["<unknown>"]=idx
    # print('idx--------------------------------', idx)
    # print('worddict--------------------------------', len(word_dict))
    return word_dict
    

In [164]:
def create_emmision_matrix(word_dict, tags_dict, tag_count, sentences):
    # words = list(brown.words())
    words_cnt = len(word_dict)
    total_tags = len(tags_dict)
    emmision_matrix = []
    
    for i in range(words_cnt):
        emmision_matrix.append([0]*total_tags)
    # print("--------------------------------------",len(word_dict))
    # print("------------------------", len(emmision_matrix))
    # print("------------------------", word_dict['<unknown>'])
    
    for sentence in sentences:
        for j in sentence:
            emmision_matrix[word_dict[j[0]]][tags_dict[j[1]]]+=1

    
    for i in range(words_cnt):
        for j in range(total_tags):
            emmision_matrix[i][j]/=tag_count[j]
    # print("------------------------", emmision_matrix[word_dict['unknown'])

    # handling missing case
    aux=[0]*len(tags_dict.keys())
    aux[tags_dict['NOUN']]=0.5
    aux[tags_dict['VERB']]=0.25
    aux[tags_dict['ADV']]=0.125
    aux[tags_dict['ADJ']]=0.125
    # print(aux)
    emmision_matrix[word_dict['<unknown>']]=aux
    

    return emmision_matrix

In [165]:
def remove_tags(sentence):
    new_sentence=[]
    for i in sentence:
        new_sentence.append(i[0])

    # print(new_sentence)

    return new_sentence

In [188]:
def HMM_logic(input, transition_matrix, emmision_matrix, tags_dict, word_dict):
    tags_output=["<start>"]
    last_prob = 1
    
    for i in range(1, len(input)):
        tag = tags_output[i-1]
        curr_ob = input[i]
        if curr_ob not in word_dict:
            curr_ob="<unknown>"

            
        max_prob = 0
        # print(emmision_matrix[word_dict[curr_ob]])
        curr_tag = ""
        for j in tags_dict:
            temp = transition_matrix[tags_dict[tag]][tags_dict[j]]*emmision_matrix[word_dict[curr_ob]][tags_dict[j]]*last_prob
            if(temp >= max_prob):
                curr_tag = j
                max_prob = temp
        last_prob = max_prob
        tags_output.append(curr_tag)
        # print(curr_tag)

    return tags_output  

In [189]:
sentences_original = brown.tagged_sents(tagset='universal')
sentences = add_start_end_tag(sentences_original)
# (tags_dict, tags_reverse, tag_count) = creat_tags_meta(sentences)
# (transition_matrix, bigram_matrix) = create_transition_matrix(sentences, tags_dict, tag_count)
# word_dict = create_word_dict(sentences)
# emmision_matrix = create_emmision_matrix(word_dict, tags_dict, tag_count, sentences)

In [198]:
def build_confusion_matrix(cm,actual,predicted,tags_dict):
    for i in range(len(actual)):
        cm[tags_dict[actual[i][1]]][tags_dict[predicted[i]]]+=1

In [199]:
# input=["<start>","do","you","know","who","is","vartul", "?", "<end>"]
# tags_output = HMM_logic(input, transition_matrix, emmision_matrix, tags_dict, word_dict)
# print(tags_output)

In [200]:
#splits data in k sets:
def k_splits (k, data):
    splits = {}
    n = len(data)//k
    for i in range(0, k):
        i = int(i)
        if(i+1 == k):
            splits[i]= data[n*i : ]
        splits[i]=data[n*i : n*(i+1)]
    return splits

In [210]:
cross_validation_set=k_splits(5,sentences)
keys=list(cross_validation_set.keys())

for i in keys:
    train_data=[]
    for k in cross_validation_set.keys():
        if k!=i:
            train_data+=cross_validation_set[k]

    test_data=cross_validation_set[i]
    # print(test_data[1:20])
    # break
    (tags_dict, tags_reverse, tag_count) = creat_tags_meta(train_data)
    confusion_matrix=[]
    for i in range(len(tag_count)):
        confusion_matrix.append([0]*len(tag_count))
    # print(tags_dict)
    (transition_matrix, bigram_matrix) = create_transition_matrix(train_data, tags_dict, tag_count)
    word_dict = create_word_dict(train_data)
    emmision_matrix = create_emmision_matrix(word_dict,  tags_dict, tag_count, train_data)
    # c=0
    for sen in test_data:
        # if c == 10:
        #     break
        # c+=1
        # print(remove_tags(sen))
        tags_output = HMM_logic(remove_tags(sen), transition_matrix, emmision_matrix, tags_dict, word_dict)
        # print(len(remove_tags(sen)),len(tags_output))
        build_confusion_matrix(confusion_matrix,sen,tags_output,tags_dict)
        
    print(confusion_matrix)

In [220]:
actual=0
count=0
for i in range(0,len(confusion_matrix)):
    actual+=confusion_matrix[i][i]
    count+=sum(confusion_matrix[i])
actual/=5
count/=5
print(actual/count)

0.9483230797820835


In [221]:
def recall(confusion_matrix):
    recall = []
    n = len(confusion_matrix)
    for i in range(0,n):
        sum = 0
        for j in range(0,len(confusion_matrix[i])):
            sum+=confusion_matrix[i][j]
        recall.append(confusion_matrix[i][i]/sum)

    return recall

In [222]:
def precision(confusion_matrix):
    precision = []
    n = len(confusion_matrix)
    for i in range(0,n):
        sum = 0
        for j in range(0,len(confusion_matrix[i])):
            sum+=confusion_matrix[j][i]
        precision.append(confusion_matrix[i][i]/sum)

    return precision

In [223]:
print("recall : ", recall(confusion_matrix))

recall :  [1.0, 0.9807788317523715, 0.9578607111372318, 0.8561136478944699, 0.9473537518193924, 0.9052294672887057, 0.9998968469552659, 0.9997384025113359, 0.8352267210408778, 0.995647807164379, 0.8195915860586519, 0.941598955310505, 0.9850746268656716, 0.1270718232044199]


In [224]:
print("precision : ", precision(confusion_matrix))

precision :  [1.0, 0.9651191353475804, 0.9133090501541048, 0.874300518134715, 0.9712353557481664, 0.8959944158075601, 1.0, 1.0, 0.918918918918919, 0.9868923179027709, 0.7654143963292228, 0.9971573448063922, 0.8734491315136477, 0.3484848484848485]
